In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.models as models
import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import PGD, FGSM


from src.models  import CifarResNet, MNIST_CNN, CIFAR_CNN
from src.helpers import evaluate_rob_accuracy, evaluate_clean_accuracy, load_model, safe_model,_evaluate_model
from src.data_loader import load_torchvision_dataset, load_imagenette
#from src.pruning import identify_layers, _evaluate_sparsity

import time

if torch.cuda.is_available() == True:
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print(device)
dtype = torch.float32

cuda:0


In [12]:
#rob_model = CifarResNet()
#rob_model.to(device)
train_loader, test_loader, attack_loader = load_torchvision_dataset('CIFAR10', data_augmentation=True)
#train_loader, test_loader = load_torchvision_dataset('CIFAR10', data_augmentation=True)
PATH='./saved-models/fast-double-vs-standard-experiment-standard-fast-training.pt'
rob_model = CifarResNet().to(device)
rob_model = load_model(rob_model, PATH)
PATH='./saved-models/cifar-resnet-100e-data_aug'
clean_model = CifarResNet().to(device)
clean_model = load_model(clean_model, PATH)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
identifying layers
identifying layers


In [13]:
evaluate_clean_accuracy(clean_model,test_loader, device)

(88.82, 0.0)

In [14]:
evaluate_clean_accuracy(rob_model,test_loader, device)

(87.93, 0.0)

In [15]:
_, rob = FGSM(clean_model, test_loader, torch.nn.CrossEntropyLoss(), 2/255, device)
rob

0.7321

In [16]:
_, rob = FGSM(clean_model, test_loader, torch.nn.CrossEntropyLoss(), 4/255, device)
rob

0.5744

In [17]:
_, rob = FGSM(clean_model, test_loader, torch.nn.CrossEntropyLoss(), 8/255, device)
rob

0.3444

In [18]:
_, rob = PGD(clean_model, test_loader, torch.nn.CrossEntropyLoss(), 7, .5/255, 2/255, device)
rob

0.1

In [19]:
_, rob = PGD(clean_model, test_loader, torch.nn.CrossEntropyLoss(), 7, 1/255, 4/255, device)
rob

0.1185

In [20]:
_, rob = PGD(clean_model, test_loader, torch.nn.CrossEntropyLoss(), 7, 2/255, 8/255, device)
rob

0.1181

In [21]:
_, rob = FGSM(rob_model, test_loader, torch.nn.CrossEntropyLoss(), 2/255, device)
rob

0.7212

In [22]:
_, rob = FGSM(rob_model, test_loader, torch.nn.CrossEntropyLoss(), 4/255, device)
rob

0.5447

In [23]:
_, rob = FGSM(rob_model, test_loader, torch.nn.CrossEntropyLoss(), 8/255, device)
rob

0.299

In [24]:
_, rob = PGD(rob_model, test_loader, torch.nn.CrossEntropyLoss(), 7, .5/255, 2/255, device)
rob

0.1

In [25]:
_, rob = PGD(rob_model, test_loader, torch.nn.CrossEntropyLoss(), 7, 1/255, 4/255, device)
rob

0.1

In [26]:
_, rob = PGD(rob_model, test_loader, torch.nn.CrossEntropyLoss(), 7, 2/255, 8/255, device)
rob

0.1006

In [10]:
def PGD(model, data_loader, criterion, steps, max_stepsize, eps, device):
    model.eval()
    advs = []
    correct = 0
    total = 0
    for i, data in enumerate(data_loader):
        inputs, labels = data
        inputs, labels =inputs.to(device), labels.to(device)
        adv_examples = inputs
        #adv_inputs = inputs
        #adv_inputs.requires_grad = True
        #perturbation = torch.zeros_like(adv_inputs, requires_grad=True).to(device)
        for step in range(steps):
            adv_examples, pert = FGSM_step(model, inputs, labels, criterion, max_stepsize, device)
            adv_examples.clamp_(-eps, eps)
            #preds = model(adv_inputs)
            #loss = criterion(preds, labels)
            #loss.backward()
            #perturbation = torch.sign(adv_inputs.grad).clamp_(-max_stepsize, max_stepsize)
            #adv_inputs = inputs + perturbation
        
        advs.append(adv_examples)
        preds = model(adv_examples)
        #pred_labels = 
        _, predicted = torch.max(preds.data, 1)
        total += len(predicted)
        #correct += (pred_labels == labels).sum().item()
        correct += (predicted == labels).sum().item()
    return advs, correct/total
        

def FGSM_step(model, inputs, labels, criterion, eps, device):

    inputs.requires_grad = True
    perturbation = torch.zeros_like(inputs, requires_grad=True).to(device)
    preds = model(inputs)
    loss = criterion(preds, labels)
    loss.backward()
    perturbation = torch.sign(inputs.grad).clamp_(-eps, eps)
    adv_examples = inputs + perturbation
    return adv_examples, perturbation
    

def FGSM(model, data_loader, criterion, eps, device):
    model.eval()
    #mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
    #mean = torch.tensor(mean).view(3,1,1).expand(3,32,32).to(device)
    #std = torch.tensor(std).view(3,1,1).expand(3,32,32).to(device)
    advs = []
    correct = 0
    total = 0
    for data in data_loader:
        inputs, labels = data
        inputs, labels =inputs.to(device), labels.to(device)
        adv_examples, perturbation = FGSM_step(model, inputs, labels, criterion, eps, device)
        advs.append(adv_examples)
        preds = model(adv_examples)
        #pred_labels = 
        _, predicted = torch.max(preds.data, 1)
        total += len(predicted)
        #correct += (pred_labels == labels).sum().item()
        correct += (predicted == labels).sum().item()

    
    return advs, correct/total

In [ ]:
a = 0

In [ ]:
for data in attack_loader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    print(accuracy(fmodel, images, labels))

In [ ]:

#model = models.resnet18(pretrained=True).eval()
model = model.eval()
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel = PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)
#fmodel = PyTorchModel(model, bounds=(0, 1))

    # get data and test the model
    # wrapping the tensors with ep.astensors is optional, but it allows
    # us to work with EagerPy tensors in the following
#images, labels = next(iter(attack_loader))[0].to(device), next(iter(attack_loader))[1].to(device)
clean_acc = accuracy(fmodel, images, labels)
print(f"clean accuracy:  {clean_acc * 100:.1f} %")

    # apply the attack
attack = FGSM()
epsilon = 0.06
raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilon)

    # calculate and report the robust accuracy (the accuracy of the model when
    # it is attacked)
robust_accuracy = 1 - success.float32().mean(axis=-1)
print("robust accuracy for perturbations with")


In [ ]:
robust_accuracy = 1 - success.float32().mean(axis=-1)
robust_accuracy

In [ ]:
diffs = []
for i, img in enumerate(clipped_advs):
    diffs.append(img - images[i])
len(diffs)

In [ ]:
images = (ep.astensor(images))
raw_advs = ep.astensor(raw_advs)

In [ ]:
model(images.to(device))

In [9]:
def load_torchvision_dataset(dataset, batchsize=512, data_augmentation=False):
    if data_augmentation == True:
        train_transforms = torchvision.transforms.Compose([
            #torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.RandomRotation(20),
            torchvision.transforms.Resize(40),
            torchvision.transforms.RandomResizedCrop(32),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),        
        ])
    if data_augmentation == False:
        train_transforms = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),        
        ])
    val_transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
    
    
    if dataset == 'MNIST':
        train = torchvision.datasets.MNIST('./data', train=True, transform=torchvision.transforms.ToTensor(), download=True)
        test = torchvision.datasets.MNIST('./data', train=False, transform=torchvision.transforms.ToTensor(), download=True)
        attack = torchvision.datasets.MNIST('./data', train=False, transform=torchvision.transforms.ToTensor(), download=True)
    if dataset == 'CIFAR10':
        train = torchvision.datasets.CIFAR10('./data', train=True, transform=train_transforms, download=True)
        test = torchvision.datasets.CIFAR10('./data', train=False, transform=val_transforms, download=True)
        attack = torchvision.datasets.CIFAR10('./data', train=False, transform=torchvision.transforms.ToTensor(), download=True)
    train_loader = torch.utils.data.DataLoader(
        train,
        batch_size=batchsize,
        shuffle=True,
    )
    test_loader = torch.utils.data.DataLoader(
        test,
        batch_size=batchsize,
        shuffle=True,
    )
    attack_loader = torch.utils.data.DataLoader(
        attack,
        batch_size=batchsize,
        shuffle=True,
    )
    return train_loader, test_loader, attack_loader